In [1]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
import astrodendro
import os
from astropy import wcs
from astrodendro import Dendrogram, pp_catalog
from astrodendro.analysis import PPStatistic
from astropy.table import Table, hstack, Column
from astropy.utils.console import ProgressBar
import reproject
import pyregion
import astropy.wcs as wcs

print(np.__version__)

1.11.3


In [2]:
# Image information and assumptions
distance        = 8340. # distance to GC; Reid et al. 2014
#Temp            = 20.0
Wave            = (3.0e8/226.e9)
Wave0           = 1.3e-3
k0              = 0.899
nu              = 3.e08/Wave
nu0             = 3.e08/Wave0
beta            = 1.75
Kappag2d        = k0*((nu/nu0)**beta)
g2d             = 100.0
Kappa           = Kappag2d / g2d
mu              = 2.8 # express everything in H2

dustTemp = 20

# Constants
G = 6.67408e-11
msun = 1.989e33
mh = 1.6737236e-27
pc2cm = 3.08567758e18
as2persr = 4.25e10 
percm2perm = 1.0e6
JyperSr_to_JyperPix = (3.0462*(10**-4))*((0.00013888888888)**2)
JyperSr_to_Jyperassqr = ((np.pi*180.)**(-2))*(3600**(-1))
hplanck = 6.63e-34
clight = 2.99792e8
kboltzmann = 1.381e-23
sin1yr = 3.15569e7
arcsec2pc = distance/((360./(2.*np.pi))*60.*60.)

### Define physical property functions

def planck_wave( Wave, Temp ):

    # Planck function using wavelength

    planck_conv_wave = 1.e-26 * clight / Wave**2.0

    planck = ((2.0*hplanck*clight**2.0)/(Wave**5.0))*(1.0/(np.exp((hplanck*clight)/(Wave*kboltzmann*Temp))-1.0))
    planck = planck/planck_conv_wave

    return planck

def mass_calc_submm( Wave, Temp, Kappa, Integrated_Flux, Obj_Dist ):

    Obj_Dist = Obj_Dist * pc2cm
    #from planck_func import planck_wave
    B = planck_wave( Wave, Temp )
    Mass = (Obj_Dist**2. * Integrated_Flux) / ( Kappa * B )
    Mass = Mass / msun
    return Mass

def column_density(Wave, Temp, Kappa, Flux, mu):

    #from planck_func import planck_wave

    B = planck_wave( Wave, Temp )

    N = Flux / (mu * (mh*1.e3) * Kappa * B)

    return N

def number_density_sphere_pc( Mass_sol, Radius_pc, mu ):

    # This subroutine accepts mass in solar masses and radius in pc and calculates the number density.

    Mass = Mass_sol * (msun/1000.0)
    Radius = Radius_pc * (pc2cm/100.0)

    n = Mass / (((4. / 3.)*np.pi) * mu * mh * Radius**3.0)

    # Convert to particles per cubic centimetre

    n = n / percm2perm

    return n

def mass_density_sphere( Mass_sol, Radius_pc ):

    # This subroutine accepts mass in solar masses and radius in pc and calculates the mass density.

    Mass = Mass_sol * (msun/1000.0)
    Radius = Radius_pc * (pc2cm/100.0)

    rho = Mass / (((4. / 3.)*np.pi) * Radius**3.0)

    return rho

def tff_spherical( number_density, mu ):

    # Accepts a number density in units of particles per cubic centimetre and derives the free fall time in yrs

    mass_density = mu * mh * number_density * percm2perm

    tff = np.sqrt( (3. * np.pi) / (32. * G * mass_density) )

    tff = tff / sin1yr # free-fall time in years

    return tff

In [3]:
### define function for initial catalog table

def run_characterization(Target, DStructure, final_noisemap,pp,pm,filesuffix, path):
    InputMap = fits.open(Target+'.fits')[0]
    
    HeaderMod = InputMap.header.copy()
    #print HeaderMod

    PixelAreaArcsec = 3600. * abs(InputMap.header['CDELT1']) * 3600. * abs(InputMap.header['CDELT2'])
    #Compile Metadata
    Metadata = {}
    Metadata['data_unit'] = u.Jy / u.sr
    Metadata['spatial_scale'] =  PixelAreaArcsec**0.5 * u.arcsec

    Metadata['wcs'] = wcs.WCS(header=HeaderMod).celestial
    #Actually build the catalog from the initial dendrogram
    Catalogue = astrodendro.pp_catalog(DStructure, Metadata)
    #Customize catalog with units and coordinate system
    Catalogue['x_cen'].unit = u.deg
    Catalogue['y_cen'].unit = u.deg
    Catalogue.rename_column('_idx', 'index')
    Catalogue.rename_column('flux', 'flux_integrated')
    Catalogue.rename_column('x_cen', 'glon')
    Catalogue.rename_column('y_cen', 'glat')
    Catalogue['flux_integrated'].unit = u.Jy
    Catalogue['flux_integrated'] = Catalogue['flux_integrated']
    Catalogue['r_eff'] = (Catalogue['area_exact']/np.pi)**0.5
    Catalogue['r_eff'].unit = u.arcsec
    Catalogue['glon'].unit = u.deg
    Catalogue['glat'].unit = u.deg
    
    # add a 'noise' column to the catalog
    keys = ['testflux_integrated', 'flux_integrated_bgsub','min_flux_integrated', 'noise', 'is_leaf', 'peak_cont_flux', 'min_cont_flux', 'mean_cont_flux','npix']
    columns = {k:[] for k in (keys)}
    
    for ii, row in enumerate(ProgressBar(Catalogue)):
        structure = dend[row['index']]
        assert structure.idx == row['index'] == ii
        dend_inds = structure.indices()
        columns['noise'].append(final_noisemap[dend_inds].mean())
        columns['is_leaf'].append(structure.is_leaf)
        peakflux = data[dend_inds].max()
        testflux = np.nansum(data[dend_inds])
        testflux_integrated = testflux*JyperSr_to_JyperPix
        minflux = data[dend_inds].min()
        numpix = np.shape(dend_inds)[1]
        backgroundflux = minflux*numpix*JyperSr_to_JyperPix
        columns['flux_integrated_bgsub'].append(Catalogue['flux_integrated'][ii] - backgroundflux)
        columns['peak_cont_flux'].append(peakflux)
        columns['min_cont_flux'].append(data[dend_inds].min())
        columns['mean_cont_flux'].append(data[dend_inds].mean())
        columns['npix'].append(numpix)
        columns['min_flux_integrated'].append(backgroundflux)
        columns['testflux_integrated'].append(testflux_integrated)
        
    for k in columns:
        if k not in Catalogue.keys():
            Catalogue.add_column(Column(name=k, data=columns[k]))
    
    Catalogue['peak_cont_flux'].unit = u.Jy / u.sr
    Catalogue['mean_cont_flux'].unit = u.Jy / u.sr
    Catalogue['mean_cont_flux'].unit = u.Jy / u.sr
    
    #pruning
    print("Pruning...")
    cat_mask = (Catalogue['is_leaf'] &
                (Catalogue['peak_cont_flux']>pp*Catalogue['noise']) &
                (Catalogue['mean_cont_flux']>pm*Catalogue['noise']))
    pruned_ppcat = Catalogue[cat_mask]
    mask = dend.index_map.copy()
    for ii in ProgressBar(list(range(len(Catalogue)))):
        if ii not in pruned_ppcat['index']:
            mask[mask == ii] = -1
    print(mask.shape)
    print(mask)
    outf = fits.PrimaryHDU(data=mask, header=HeaderMod)
    print(outf)
    print("Writing mask file...")
    outf.writeto(path+'dendrogram_mask_pruned'+filesuffix+'.fits', overwrite=True)
    
    print("Writing region file...")
    with open(path+"raw_dendrogram_cores"+filesuffix+".reg", 'w') as fh:
        fh.write("galactic\n")
        for row in pruned_ppcat:
            fh.write("ellipse({glon}, {glat}, {major_sigma}\", "
                     "{minor_sigma}\", {position_angle}) # text={{{index}}}\n"
                     .format(**dict(zip(row.colnames, row))))
        
    print("Region file complete!")
    
    print("Writing data table...")        
    OutputPath = path + 'catalog'+'_datatab'+filesuffix+'.fits'
    if os.path.isfile(OutputPath):
        os.remove(OutputPath)
    pruned_ppcat.write(OutputPath,
                    format='fits')
    print("Data table written!")
    

In [12]:
### RUN THE CATALOG

JyperSr_to_JyperPix = (3.0462*10**(-4))*((0.00013888888888)**2)

rms = 1*10**7
dend_min_v = 3
dend_min_d = 1
dend_min_pix = 50
prune_min_peak = 6
prune_min_mean = 2

date = '_8-6-19'
#directory = '/Users/hph/Dropbox/CMZoom_Data/continuum_images/'
directory = '/Users/hph/cmzoom_catalog/add_SgrB2sw/'
target = directory+'G0.714-0.100.continuum.pbcor'
filesuffix = 'SgrB2sw_acc'


sma = fits.open(directory + 'G0.714-0.100.continuum.pbcor.fits')
data = sma[0].data / JyperSr_to_JyperPix
HeaderMod = sma[0].header

#print data.shape

mywcs = wcs.WCS(HeaderMod)

In [13]:
print( "Running dendrogram...")
dend = astrodendro.Dendrogram.compute(data, min_value=float(dend_min_v)*rms, min_delta=float(dend_min_d)*rms,
                                      min_npix=dend_min_pix,
                                      wcs=mywcs)
print("Dendrogram complete!")
#Save the dendrogram to the working directory (temporary)
dend.save_to(directory+'/prototype_dendrogram'+filesuffix+'.fits')

noisemap = fits.open(directory+'G0.714-0.100_noisemap.fits')
noisemap_data = noisemap[0].data / JyperSr_to_JyperPix
print( "Running characterization...")
#Perform pruning and save catalog to working directory (temporary)
run_characterization(target,dend,noisemap_data,
                     float(prune_min_peak),float(prune_min_mean),filesuffix,path=directory)
print("Characterization complete!")



Running dendrogram...


/Users/hph/anaconda/envs/cmzoom3/lib/python3.4/site-packages/astrodendro/dendrogram.py:177: RuntimeWarning: invalid value encountered in greater
  keep = self.data > min_value


Dendrogram complete!
Running characterization...
Computing catalog for 1666 structures

[>                                        ] 0%
[>                                        ] 0%
[>                                        ] 0%
[>                                        ] 0%
[>                                        ] 0%
[>                                        ] 0%


[>                                        ] 0%
[>                                        ] 0%
[>                                        ] 0%
[>                                        ] 0%
[>                                        ] 0%
[>                                        ] 0%
[>                                        ] 0%
[>                                        ] 0%
[>                                        ] 0%
[>                                        ] 0%
[>                                        ] 1%
[>                                        ] 1%
[>                                        ] 1%
[>                                        ] 1%
[>                                        ] 1%
[>                                        ] 1%
[>                                        ] 1%
[>                                        ] 1%
[>                                        ] 1%
[>                                        ] 1%
[>                                        ] 1%
[>          

In [ ]:
### add columns to catalog table
